In [1]:
import requests
import json
import time
import codecs
import sys
import time
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
import json
import os
import datetime
import re
import random, copy

In [2]:
api_key = ''

In [3]:
UTF8Writer = codecs.getwriter('utf8')
sys.stdout = UTF8Writer(sys.stdout)
url_string = "http://api.meetup.com//"

In [ ]:
#Fetching complete category,cities,topic_categories information using meetup.api package
import meetup.api
client = meetup.api.Client(api_key,overlimit_wait=True)
a = client.GetCategories()
#Categories
categories_df = pd.read_json(json.dumps(a.results))
categories_df.to_csv('categories.csv', index = False)
#Cities
cities = client.GetCities(country='US',state=)
cities_df = pd.read_json(json.dumps(cities.results))
cities_df.to_csv('cities.csv',index=False)
#Topic_Categories
topic_categories = client.GetTopicCategories()
topic_catg_df = pd.read_json(json.dumps(topic_categories.results))
topic_catg_df.to_csv('topic_categories.csv',index=False)

In [ ]:
# Read Categories File
df = pd.read_csv('categories.csv')
catg_id = df.id #you can also use df['column_name']

In [ ]:
#Function to fetch group
def getgroups(catg_id,off_val):
    group = client.GetGroups(category_id = catg_id,offset = off_val)
    result = group.results
    RES = json_normalize(result)
    return RES

In [ ]:
#Create groups files for every catrgory
for i in range(len(catg_id)):
    groups = pd.DataFrame([])
    for j in range(10000):
        result = getgroups(catg_id[i],j)
        if len(result) != 0:
            groups = groups.append(result)
        else:
            break
    groups.to_csv('groups_'+ str(catg_id[i]) +'.csv',index=False,encoding='utf-8')

In [ ]:
# Call the api
def get_results(string, params):
    request = requests.get(string,params=params)
    return request.json()

In [ ]:
#Fetch unique groups 
cwd = os.getcwd()
group_id_extract = pd.DataFrame([])
dat_list = []
for i in catg_id:
    data = pd.read_csv(cwd+'/group_data/groups_'+str(i)+'.csv',usecols=[13,4], index_col = False, header=0)   
    dat_list.append(data)
group_id_extract = pd.concat(dat_list)
group_id_new = group_id_extract[group_id_extract['country']=='US']['id']
group_id_US = group_id_new.drop_duplicates().reset_index(drop=True)
unique_groups = np.array(group_id_US)

In [ ]:
#Function to fetch members data
def get_members( group_id, index,num_groups ):
    result =[]
    prev=[]
    for offset in range(10000):
        count = 0
        while True:
            params = {"sign":"true", "key":api_key, "page":200, "offset":offset, "group_id":group_id}
            try:
                time.sleep(0.1)
                temp = get_results(url_string + "2/members", params)
            except ValueError, ChunkedEncodingError:
                continue
            break
        if len(temp['results']) != 0:
            if prev == temp['results']:
                value = pd.DataFrame(pd.io.json.json_normalize(result))
                value['group_id']=[group_id] *len(value)
                
                value.to_csv("./member_data/member_"+str(group_id)+'.csv', encoding = 'utf-8',sep=str(","))
                return None
            result = result + temp['results']
            sys.stdout.write('\r' + "Processed: " + str(len(result)) + " events for group number " + str(group_id) +\
                             ". Progress: "+ str(index) +'/'+str(num_groups))
            sys.stdout.flush()
            prev = copy.deepcopy(temp['results'])
        else:
            break
    sys.stdout.write('\r' + " "*150 + '\r')
    value = pd.DataFrame(pd.io.json.json_normalize(result))
    value['group_id']=[group_id] *len(value)
    value.to_csv("./member_data/member_"+str(group_id)+'.csv', encoding = 'utf-8',sep=str(","))

In [ ]:
#Fetch members and extract to csv
num_groups = len(unique_groups)
for index, group_id in enumerate(unique_groups):
    (get_members(group_id,index, num_groups))

In [ ]:
#Function to fetch events
def getevents(group_id,off_val):
    event = client.GetEvents(group_id = group_id, offset = off_val)
    result = event.results
    RES = json_normalize(result)
    return RES

In [ ]:
#Fetch events and extract to csv
events = pd.DataFrame([])
for i in range(len(unique_groups)):
    for j in range(10000):
        result = getevents(unique_groups.id[i], j)
        if len(result) != 0:
            events = events.append(result)
        else:
            break
events.to_csv('events.csv',index=False,encoding='utf-8')

In [ ]:
#Function to fetch venues
def getvenues(group_id,off_val):
    venue = client.GetVenues(group_id = group_id, offset = off_val)
    result = venue.results
    return result
    RES = json_normalize(result)
    return RES

In [ ]:
#Fetch venues and extract csv
cwd = os.getcwd()
for index, row in data_US_idx.iteritems():
    group_id = row
for index, group_id in enumerate(unique_groups):
    for offset in range(10000):
        try: 
            return_value = getvenues(group_id,offset)
            time.sleep(0.05)
        except:
            time.sleep(3610)
            return_value = getvenues(group_id,offset)
            print("Exception was raised!")
            pass
        if len(return_value) > 0:
            temp_df = pd.DataFrame(return_value)
            temp_df['group_id'] = [group_id]* len(temp_df)
            venues = venues.append(temp_df,ignore_index=True)
        else:
            break
    if index % 100 ==0:
        venues.to_csv(cwd+'/venue_data/venues_'+str(i)+ '_'+ str(index) + '.csv', index = False, encoding = 'utf-8')
        venues = pd.DataFrame()
venues.to_csv(cwd+'/venue_data/venues_final.csv', index = False, encoding = 'utf-8')

In [ ]:
#List of topics_categories
topics = ['art','spirituality','literature','business-networking','professional-networking','dance','dancing','education','fashion','Fashion Industry','movies','Watching movies','wine-and-food','wine-and-food-pairing','games','wellness','craftswap','crafts','language','Language & Culture','lgbtfriends','LGBT','community','music','outdoors','parents','pet-animals','pets','photo','photography','social','sports','Sports and Recreation','newtech','New Technology','writing']

In [ ]:
#Function to fetch topics
def gettopics(topic_name,off_val):
    topics = client.GetTopics(search = topic_name,offset = off_val)
    result = topics.results
    RES = json_normalize(result)
    return RES

In [ ]:
#Fetch topics and extract csv
for i in range(len(topics)):
    topics = pd.DataFrame([])
    for j in range(10000):
        result = gettopics(str(topics[i]),j)
        if len(result) != 0:
            topics = topics.append(result)
        else:
            break
    topics.to_csv('./topics_data/topic_'+ str(i) +'.csv',index=False,encoding='utf-8')